In [2]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, models
import numpy as np
import os
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras import layers


First thing first we need to define some of the constraints of the problem.

First off there are only 2 classes (either cat and dog).

We can use the sigmoid activation function since we only have two classes

The pictures are in color (RGB)

We also have two folders one for cats and the other for dogs.

This means when we make the full dataset, we will have to turn all the pictures into arrays. ensure we keep the labels.
Then we need to make sure that all the cats and dogs datasets can be combined and that you choose the data randomly (with test_train_split)



In [3]:
data_file_path = 'C:/Users/franc/Documents/neural_nets/resnet_cats_vs_dogs/PetImages'

In [4]:
train_dataset = keras.preprocessing.image_dataset_from_directory(
    data_file_path,
    labels='inferred',
    label_mode='categorical',
    batch_size = 64,
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset='training',
    image_size=(224,224)
)

validation_dataset = keras.preprocessing.image_dataset_from_directory(
    data_file_path,
    labels='inferred',
    label_mode = 'categorical',
    batch_size = 64,
    shuffle = True,
    seed = 42,
    validation_split = 0.2,
    subset = 'validation',
    image_size = (224,224)

)


Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


I will now make the train and validation split. 

Kindly remember that this will be an image pipeline not a dataframe pipeline since turning the data into an df pipeline would flatten the data

In [5]:
type(validation_dataset)

tensorflow.python.data.ops.prefetch_op._PrefetchDataset

In [6]:
autotune = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=autotune)
validation_dataset = validation_dataset.prefetch(buffer_size=autotune)

I then want to do some augmentation on the train part of the data. 

So that the finetuning can be more effective

In [7]:
data_augment = keras.Sequential([
    layers.RandomRotation(0.05),        # ~±9°
    layers.RandomZoom(0.1),
    layers.RandomFlip("horizontal"),
    layers.RandomContrast(0.1),
])

# Apply during training only:
# augmented_train_ds = train_dataset.map(lambda x, y: (data_augment(x, training=True), y))
# augmented_train_ds = augmented_train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

In [8]:
#Apply during training only:
augmented_train_ds = train_dataset.map(lambda x,y:(data_augment(x,training=True),y))

Importing the Resnet50 model

The reason why I chose Resnet50 is because it can be imported through tensorflow unlike some other smaller variants of resnet that can only be used with the transformers library.

In [9]:
base_model = keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

I am going to call a function for prepreocessing the input

In [14]:
preprocess_input = keras.applications.resnet.preprocess_input

I will first attempt to use the frozen model to make predictions using the pictures that we have

In [10]:
#rescaling
rescale = keras.layers.Rescaling(1/255)

In [11]:
base_model.summary()

Model: "resnet50"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_1[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 23,534,592 (89.78 MB)

 Non-trainable params: 53,120 (207.50 KB)

To generate predictions I sill have to add a classification head to the model.

I will use the globalaveragepooling2s layer to flatten the data.

Then I will make a fully connected layer to make the predictions.

Remember I will only have two classes

In [15]:
inputs = keras.Input(shape=(224,224,3))
x= preprocess_input(x)
x = base_model(inputs,training=False)
x= layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(2,activation='sigmoid')(x)

model = keras.Model(inputs,outputs)

In [16]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │         4,098 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,591,810 (90.00 MB)

 Trainable params: 23,538,690 (89.79 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [23]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),loss=keras.losses.CategoricalCrossentropy(),metrics=[keras.metrics.Accuracy()])

In [ ]:
model_frozen = model.fit(training_dataset,validation_dataset,epochs=10,shuffle=True,batch_size=64)

In [ ]:
acc = model_frozen.history['accuracy']
val_acc = model_frozen.history['val_accuracy']

loss = model_frozen.history['loss']
val_loss = model_frozen.history['val_loss']


In [ ]:
plt.figure(figsize=(8,8))
plt.subplot(2,1,1)
plt.plot(loss,label='loss')
plt.plot(val_loss,label='val_loss')
plt.legend()
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.title('Loss vs Epochs')

plt.subplot(2,1,2)
plt.plot(acc,label='accuracy')
plt.plot(val_acc,label='val_acc')
plt.legend()
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.title('Accuracy vs Epochs')

I will now finetune a model by training its last 20 layers.